# Blender Parameterization Helper Testing


## Imports & Setup



## Load & Parse Code



### Import Test Code


In [34]:

test_code = """
###############
### Imports ###
###############

import bpy
import math


############
### Prep ###
############

# Clear Scene
bpy.ops.object.select_all(action='DESELECT')
bpy.ops.object.select_by_type(type='MESH')
bpy.ops.object.delete()

##################
### Parameters ###
##################

weave_name = "Orbital Vipera Beerus Lace"

large_ring_minor = 0.18
large_ring_major = 1.42

small_ring_minor = 0.17
small_ring_major = 1.00

dist_1 = 1.50
dist_2 = 0.30
dist_3 = 1.20

num_units = 12

angle_1 = math.radians(73)
angle_2 = math.radians(164)
angle_3 = math.radians(90)


##################
### Model Code ###
##################

# Initial Small Ring
bpy.ops.mesh.primitive_torus_add(
    align="WORLD",
    location=(0,0,0),
    rotation=(angle_1,0,0),
    major_radius=small_ring_major,
    minor_radius=small_ring_minor,
    major_segments=48,
    minor_segments=12,
)

for i in range(num_units):
    base_dist = dist_1 * (i+1)
    
    mod = -1 if i % 2 == 0 else 1

    # Vipera Small Ring
    bpy.ops.mesh.primitive_torus_add(
        align="WORLD",
        location=(base_dist,0,0),
        rotation=(mod * angle_1,0,0),
        major_radius=small_ring_major,
        minor_radius=small_ring_minor,
        major_segments=48,
        minor_segments=12,
    )

    # Lace Small Ring
    if i != 0 and i != num_units-1:
        bpy.ops.mesh.primitive_torus_add(
            align="WORLD",
            location=(base_dist,-mod * dist_3,0),
            rotation=(angle_3,0,0),
            major_radius=small_ring_major,
            minor_radius=small_ring_minor,
            major_segments=48,
            minor_segments=12,
        )

    # Large Ring
    bpy.ops.mesh.primitive_torus_add(
        align="WORLD",
        location=(base_dist,mod * dist_2,0),
        rotation=(0,angle_2,0),
        major_radius=large_ring_major,
        minor_radius=large_ring_minor,
        major_segments=48,
        minor_segments=12,
    )

# Final Small Ring
mod = -1 if num_units % 2 == 0 else 1
base_dist = dist_1 * (num_units + 1)
bpy.ops.mesh.primitive_torus_add(
    align="WORLD",
    location=(base_dist,0,0),
    rotation=(mod * angle_1,0,0),
    major_radius=small_ring_major,
    minor_radius=small_ring_minor,
    major_segments=48,
    minor_segments=12,
)
"""



### Split Code By Section


In [36]:

# Set up section containers
section_mapping = {
    "Imports":   "",
    "Parameters": "",
    "Model Code": "",
}

# Initialize section tracking variable
current_section = None

# Itterate over lines to get sections of interest
for line in test_code.split('\n'):

    # Check for section headers
    if ("### " in line) and (" ###" in line):
        current_section = line.replace("#","").strip()

    # Itterate over lines in a target section
    if current_section in section_mapping:

        # Skip useless lines
        if line.strip() == "": continue # empty lines
        if line.startswith("#"): continue # comments

        # Add line to the current section
        section_mapping[current_section] += line + "\n"



### Parse Parameters


In [ ]:

parameters = dict()

# Parse parameters section lines for name
for line in section_mapping["Parameters"].split('\n'):
    if "=" in line:

        # Get Values from lines
        split_vals = line.split("=")
        name, value = map(str.strip, split_vals)

        # Check type of value
        value_type = None
        if '"' in  value or "'" in value:
            value_type = "str"
        elif value.replace('.', '', 1).isdigit():
            value_type = "number"
        elif "radians" in value:
            value_type = "angle"
            value = value.replace("radians(", "").replace(")", "").replace("math.", "").strip()
        else:
            value_type = "Unknown"

        parameters[name] = [value, value_type]


{'weave_name': ['"Orbital Vipera Beerus Lace"', 'str'],
 'large_ring_minor': ['0.18', 'number'],
 'large_ring_major': ['1.42', 'number'],
 'small_ring_minor': ['0.17', 'number'],
 'small_ring_major': ['1.00', 'number'],
 'dist_1': ['1.50', 'number'],
 'dist_2': ['0.30', 'number'],
 'dist_3': ['1.20', 'number'],
 'num_units': ['12', 'number'],
 'angle_1': ['73', 'angle'],
 'angle_2': ['164', 'angle'],
 'angle_3': ['90', 'angle']}

In [31]:

a, b = map(str.strip,"a = 1".split("="))

print(a, b)


a 1



## Update Template



### Update Code



### Customize Template



### Add Parameters To Template



### Add Updated Code To Template



## Display Updated Code
